In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
from keras.applications.vgg16 import VGG16, preprocess_input
from skimage.transform import resize
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import normalize

In [ ]:
# Liste für Trainingsbilder
trainImg = []

# Pfad zum Ordner mit den Bildern
folder_path = './PraktikumCV/ASL_Alphabet_Dataset/asl_alphabet_train/A/'

# Durchsuche alle Unterordner nach .jpg und .jpeg Dateien und füge sie zur Liste hinzu
for img_path in glob.glob(folder_path + '/*.jp*g', recursive=True):
    trainImg.append(resize(imread(img_path), (224, 224)))

# Konvertieren der Bilder in eine 2D-Array-Darstellung
image_array = np.array([img.flatten() for img in trainImg])

# Ausgabe der Anzahl der gefundenen Bilder
print(f"Anzahl der gefundenen Bilder: {len(trainImg)}")

In [ ]:
# Laden des VGG16-Modell, ohne die oberste Schicht (ohne Klassifizierungsschicht)
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Funktion zur Feature-Extraktion mit VGG16
def extract_features(img_array):
    img_array = preprocess_input(img_array)
    features = vgg16_model.predict(img_array)
    return features.reshape((len(img_array), -1))

# Feature-Extraktion mit VGG16
vgg16_features = extract_features(np.array(trainImg))

In [ ]:
# Features normalisieren
normalized_features = StandardScaler().fit_transform(vgg16_features)

# KDTree erstellen
kdtree = KDTree(normalized_features, metric='euclidean')

# Festlegung der Entfernungsschwelle
distance_threshold = 0.28

# Nachbarn für jeden Punkt finden
neighbors = kdtree.query_radius(normalized_features, distance_threshold)

# Cluster-Labels erstellen
labels = np.zeros(len(normalized_features), dtype=int)
current_label = 1

for i, neighbor_list in enumerate(neighbors):
    if labels[i] == 0:  # Punkt noch nicht in einem Cluster
        labels[neighbor_list] = current_label
        current_label += 1

# Berechnung der Centroids der Cluster
cluster_centroids = []
for cluster_label in np.unique(labels):
    cluster_points = normalized_features[labels == cluster_label]
    cluster_centroid = np.mean(cluster_points, axis=0)
    cluster_centroids.append(cluster_centroid)

cluster_centroids = np.array(cluster_centroids)

# Eindeutige Bilder basierend auf den Clustern extrahieren
unique_images = [trainImg[i] for i in np.unique(labels) if i != -1]

# Indizes der Centroids in den ursprünglichen Bildern finden
centroid_indices = []
for centroid in cluster_centroids:
    nearest_neighbor_distance, nearest_neighbor_index = kdtree.query(centroid.reshape(1, -1), k=1)
    centroid_indices.append(nearest_neighbor_index[0][0])

# Bilder der Centroids extrahieren
centroid_images = [trainImg[i] for i in centroid_indices]

# Speicherpfad für eindeutige Bilder
output_folder = './PraktikumCV/ASL_unique/A/'
os.makedirs(output_folder, exist_ok=True)  # Ordner erstellen, falls er nicht existiert

# Batch-Verarbeitung und direktes Speichern
for img_index, output_path in zip(centroid_indices, [os.path.join(output_folder, f'unique_image_{i}.jpg') for i in range(len(centroid_indices))]):
    img = trainImg[img_index]
    plt.imshow(img)
    plt.axis('off')
    imsave(output_path, img)
    plt.close()

print("Eindeutige Bilder (Centroids) wurden im Zielordner gespeichert:", output_folder)